In [1]:
import zeep
import numpy as np
import math
import time
import sys
import shutil

np.set_printoptions(threshold=sys.maxsize)

C:\Users\USER\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\USER\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
client = zeep.Client('http://turbulence.pha.jhu.edu/service/turbulence.asmx?WSDL')
ArrayOfFloat = client.get_type('ns0:ArrayOfFloat')
ArrayOfArrayOfFloat = client.get_type('ns0:ArrayOfArrayOfFloat')
SpatialInterpolation = client.get_type('ns0:SpatialInterpolation')
TemporalInterpolation = client.get_type('ns0:TemporalInterpolation')
token = "uk.ac.imperial.george.dixon-dray17-29581e35"

In [3]:
def get_points_test(DNS_nx = 1024, new_domain_nx = 1020, filter_width_n = 5, crop = 4):
    start_time = time.time()
    dataset = "isotropic1024coarse"

    # Database Domain Size
    DNS_x_min = 0
    DNS_x_max = 2*np.pi
    DNS_spacing = DNS_x_max / DNS_nx
    
    print("DNS Spacing:", DNS_spacing)
    
    sample_nx = new_domain_nx / crop / filter_width_n
    print("Number of cells: " + str(sample_nx))
    if (sample_nx % 1) == 0:
        sample_nx = int(sample_nx)
    else:
        print("get_points_test error: The inputs do not give a mesh with an integer number of cells")
        return 0

    # Sample Domain Size
    filter_spacing = filter_width_n * DNS_spacing
    sample_x_min = math.ceil(filter_width_n/2) * DNS_spacing
    sample_x_max = sample_x_min + ((sample_nx-1) * filter_spacing)

    print("filter_spacing:", filter_spacing)

    x = np.linspace(sample_x_min, sample_x_max, sample_nx)
    print("x[1]-x[0]:", x[1]-x[0])
    y = x
    z = x
    X,Y,Z = np.meshgrid(x,y,z)
    X = X.ravel()
    Y = Y.ravel()
    Z = Z.ravel()

    nnp=5 #number of points
    points=np.random.rand(nnp,3)

    points = np.zeros((sample_nx**3,3))
    points[:,0] = Z
    points[:,1] = X
    points[:,2] = Y


    # convert to JHTDB structures
    x_coor=ArrayOfFloat(points[:,0].tolist())
    y_coor=ArrayOfFloat(points[:,1].tolist())
    z_coor=ArrayOfFloat(points[:,2].tolist())
    point=ArrayOfArrayOfFloat([x_coor,y_coor,z_coor]);
    
    
    return point, filter_spacing

In [4]:
DNS_nx = 1024
new_domain_nx = 1008
filter_width_n = 18
crop = 1
point, filter_spacing = get_points_test(DNS_nx, new_domain_nx, filter_width_n, crop)

DNS Spacing: 0.006135923151542565
Number of cells: 56.0
filter_spacing: 0.11044661672776616
x[1]-x[0]: 0.11044661672776615


In [5]:
def getFields(point, filter_spacing, field = "u"):
    start_time = time.time()
    
    t = 0
    filterwidth = filter_spacing
    spacing = filter_spacing #spacing is only used in GetBoxFilterGradient, but always provide it.
    return_field = client.service.GetFilter_Python("GetBoxFilter",token,"isotropic1024coarse", field, 
                                           t, filterwidth, SpatialInterpolation("None"), point, spacing)

    number_of_component = 3
    return_field = np.array(return_field).reshape((-1, number_of_component))
    if field == "p":
        return_field = return_field[:,:1]
    
    print("Found " + field + " field")
    print("--- %s seconds ---" % (time.time() - start_time))
    print(np.shape(return_field))
    
    return return_field

In [6]:
u = getFields(point, filter_spacing, field = "u")
p = getFields(point, filter_spacing, field = "p")

Found u field
--- 43.79325485229492 seconds ---
(175616, 3)
Found p field
--- 44.7305862903595 seconds ---
(175616, 1)


In [9]:
# test_array = np.array([[0.5,0.6,0.7],[1,2,3],[7,8,9],[10,15,17]])
# test_array = np.array([[0.5],[1],[7],[10]])
def createOFfile(fieldvalues, filter_width_n, new_domain_nx, crop):

    if np.shape(fieldvalues)[1] == 3:
        field = 'U'
    else:
        field = 'p'

    new_file_name = field + "_" + str(filter_width_n) + "_" + str(new_domain_nx) + "_crop" + str(crop) + ".txt"
    template_file_name = field + ".txt"
    shutil.copy(template_file_name, new_file_name)
    f = open(new_file_name,"a")
    
    output_string = """"""
    n_cells = np.shape(fieldvalues)[0]
    print(n_cells)
    output_string += "\n" + str(n_cells) + "\n(\n"
    f.write(output_string)
    
    components = np.shape(test_field_initial)[1]

    field_string = fieldvalues.astype(str)
    space_array = np.full((1, n_cells), " ")
    open_bracket_array = np.full((1, n_cells), "(")
    close_bracket_array = np.full((1, n_cells), ")")

    if field == 'U':
        field_string = np.insert(field_string, 0, open_bracket_array, axis = 1)
        field_string = np.insert(field_string, 2, space_array, axis = 1)
        field_string = np.insert(field_string, 4, space_array, axis = 1)
        field_string = np.insert(field_string, 6, close_bracket_array, axis = 1)


    np.savetxt(f, field_string, delimiter="", fmt="%s")
        
    output_string = """)
    ;

    boundaryField
    {
        patch0
        {
            type            cyclic;
        }
        patch1
        {
            type            cyclic;
        }
        patch2
        {
            type            cyclic;
        }
        patch3
        {
            type            cyclic;
        }
        patch4
        {
            type            cyclic;
        }
        patch5
        {
            type            cyclic;
        }
    }


    // ************************************************************************* //
    """
    print(output_string)
    f.write(output_string)
    print('yeet')
    f.close()

In [10]:
createOFfile(u, filter_width_n, new_domain_nx, crop)
createOFfile(p, filter_width_n, new_domain_nx, crop)

175616
)
    ;

    boundaryField
    {
        patch0
        {
            type            cyclic;
        }
        patch1
        {
            type            cyclic;
        }
        patch2
        {
            type            cyclic;
        }
        patch3
        {
            type            cyclic;
        }
        patch4
        {
            type            cyclic;
        }
        patch5
        {
            type            cyclic;
        }
    }


    // ************************************************************************* //
    
yeet
175616
)
    ;

    boundaryField
    {
        patch0
        {
            type            cyclic;
        }
        patch1
        {
            type            cyclic;
        }
        patch2
        {
            type            cyclic;
        }
        patch3
        {
            type            cyclic;
        }
        patch4
        {
            type            cyclic;
        }
        patch5
        {
            

NameError: name 'output_string' is not defined

In [7]:
test_field_initial = np.array([[0.0, 0.1, 0.2], [0.3, 0.4, 0.5], [0.6, 0.7, 0.8], [0.9, 1.0, 1.1]])

In [60]:
print(test_field_initial)
n_cells = np.shape(test_field_initial)[0]
components = np.shape(test_field_initial)[1]

test_field = test_field_initial.astype(str)
print(test_field)

space_array = np.full((1, n_cells), " ")
open_bracket_array = np.full((1, n_cells), "(")
close_bracket_array = np.full((1, n_cells), ")")

test_field = np.insert(test_field, 0, open_bracket_array, axis = 1)
if components == 3:
    test_field = np.insert(test_field, 2, space_array, axis = 1)
    test_field = np.insert(test_field, 4, space_array, axis = 1)
    test_field = np.insert(test_field, 6, close_bracket_array, axis = 1)
else:
    test_field = np.insert(test_field, 2, close_bracket_array, axis = 1)

print(test_field)

np.savetxt('numpy_test.txt', test_field, delimiter="", fmt="%s")

[[0.  0.1 0.2]
 [0.3 0.4 0.5]
 [0.6 0.7 0.8]
 [0.9 1.  1.1]]
[['0.0' '0.1' '0.2']
 ['0.3' '0.4' '0.5']
 ['0.6' '0.7' '0.8']
 ['0.9' '1.0' '1.1']]
[['(' '0.0' ' ' '0.1' ' ' '0.2' ')']
 ['(' '0.3' ' ' '0.4' ' ' '0.5' ')']
 ['(' '0.6' ' ' '0.7' ' ' '0.8' ')']
 ['(' '0.9' ' ' '1.0' ' ' '1.1' ')']]
